<a href="https://colab.research.google.com/github/hrbolek/learning/blob/master/operanalyst/linearprog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Příklady

### Příklad 1

Surový materiál je dodáván v délce 7 m. Požadavek je produkovat délky 2.3 m (A), 1.9 m (B) a 0.8 m (C) v počtech A 60 ks, B 90 ks a C 320 ks. Stanovte minimální počet kusů surového materiálu.

In [21]:
delkaZdroje = 7
delkyVysledku = (2.3, 1.9, 0.8) 
poctyVysledku = (60, 90, 320)

In [23]:
def cutIt(source, results):
  '''
  Pocita mozne rozrezani materialu, 
  zdrojovy material je source, results cilove delky
  '''
  
  noyield = True # zadny vysledek nebyl vygenerovan
  for current in results:
    
    if current <= source: # rez lze provest jen tehdy, pokud je surovina delsi nez vysledek
      
      # zjistime mozne rezy ze zbytku
      for cutting in cutIt(source - current, results):
        if len(cutting) == 0:
          # rez je "nerez", vracena prazdna konfigurace
          noyield = False
          yield [current] # vysledna konfigurace rezu tedy obsahuje jen jeden prvek
          continue
        if current >= max(cutting):
          # vraceny cutting je serazen podle velikost, tedy zastupuje vsechny ekvivalentni kombinace 
          noyield = False
          yield [current, *cutting]
  if noyield:
    # zadna konfigurace rezu nebyla vracena, musime vratit prazdnou konfiguraci
    yield []

for index, possibleCut in enumerate(cutIt(delkaZdroje, delkyVysledku)):
  print(index, '.\tcut:', possibleCut, '\tΣ', sum(possibleCut))

0 .	cut: [2.3, 2.3, 2.3] 	Σ 6.8999999999999995
1 .	cut: [2.3, 2.3, 1.9] 	Σ 6.5
2 .	cut: [2.3, 2.3, 0.8, 0.8, 0.8] 	Σ 6.999999999999999
3 .	cut: [2.3, 1.9, 1.9, 0.8] 	Σ 6.8999999999999995
4 .	cut: [2.3, 1.9, 0.8, 0.8, 0.8] 	Σ 6.599999999999999
5 .	cut: [2.3, 0.8, 0.8, 0.8, 0.8, 0.8] 	Σ 6.299999999999999
6 .	cut: [1.9, 1.9, 1.9, 0.8] 	Σ 6.499999999999999
7 .	cut: [1.9, 1.9, 0.8, 0.8, 0.8] 	Σ 6.199999999999999
8 .	cut: [1.9, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8] 	Σ 6.699999999999999
9 .	cut: [0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8] 	Σ 6.3999999999999995


In [32]:
def countIt(configuration, results):
  result = {}
  for item in results:
    result[item] = 0
  for item in configuration:
    result[item] += 1
  return result

def countAll(configurations, results):
  for configuration in configurations:
    yield countIt(configuration, results)

variants = list(countAll(cutIt(delkaZdroje, delkyVysledku), delkyVysledku))

for index, possibleCut in enumerate(variants):
  print(index, '.\tcut:', possibleCut, '\tΣ', sum(map(lambda x: x[0] * x[1], possibleCut.items())))

0 .	cut: {2.3: 3, 1.9: 0, 0.8: 0} 	Σ 6.8999999999999995
1 .	cut: {2.3: 2, 1.9: 1, 0.8: 0} 	Σ 6.5
2 .	cut: {2.3: 2, 1.9: 0, 0.8: 3} 	Σ 7.0
3 .	cut: {2.3: 1, 1.9: 2, 0.8: 1} 	Σ 6.8999999999999995
4 .	cut: {2.3: 1, 1.9: 1, 0.8: 3} 	Σ 6.6
5 .	cut: {2.3: 1, 1.9: 0, 0.8: 5} 	Σ 6.3
6 .	cut: {2.3: 0, 1.9: 3, 0.8: 1} 	Σ 6.499999999999999
7 .	cut: {2.3: 0, 1.9: 2, 0.8: 3} 	Σ 6.2
8 .	cut: {2.3: 0, 1.9: 1, 0.8: 6} 	Σ 6.700000000000001
9 .	cut: {2.3: 0, 1.9: 0, 0.8: 8} 	Σ 6.4


In [38]:
for result, count in zip(delkyVysledku, poctyVysledku):
  for index, variant in enumerate(variants):
    if (variant[result] == 0):
      print('\t\t', end='')
    else:
      print('\t + ', variant[result], ' * x', index, sep='', end='')
  print('\t >=', count)
  
for index, variant in enumerate(variants):
  print(' + x', index, sep='', end='')
print(' -> min')


	 + 3 * x0	 + 2 * x1	 + 2 * x2	 + 1 * x3	 + 1 * x4	 + 1 * x5									 >= 60
			 + 1 * x1			 + 2 * x3	 + 1 * x4			 + 3 * x6	 + 2 * x7	 + 1 * x8			 >= 90
					 + 3 * x2	 + 1 * x3	 + 3 * x4	 + 5 * x5	 + 1 * x6	 + 3 * x7	 + 6 * x8	 + 8 * x9	 >= 320
 + x0 + x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 -> min


In [40]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html
from scipy.optimize import linprog
c = [1] * len(delkyVysledku)
A = [([0] * len(variants))] * len(delkyVysledku)
print(c)
print(A)

[1, 1, 1]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
